In [ ]:
from pyspark.sql.functions import current_timestamp, input_file_name

landing_path = "/FileStore/data/landing/dados/"
bronze_path = "/mnt/datalake/bronze/"

arquivos = {
    "clientes": {"formato": "csv", "path": f"{landing_path}clientes.csv"},
    "produtos": {"formato": "csv", "path": f"{landing_path}produtos.csv"},
    "vendedores": {"formato": "csv", "path": f"{landing_path}vendedores.csv"},
    "pedidos": {"formato": "csv", "path": f"{landing_path}pedidos.csv"},
    "itens_pedido": {"formato": "json", "path": f"{landing_path}itens_pedido.json"}
}

print("Iniciando a ingestão para a camada Bronze...")

for nome_tabela, info in arquivos.items():
    print(f"Processando tabela: {nome_tabela}...")
    
    if info["formato"] == "csv":
        df = spark.read.format("csv") \
                      .option("header", "true") \
                      .option("inferSchema", "true") \
                      .load(info["path"])
    elif info["formato"] == "json":
        df = spark.read.format("json") \
                      .option("inferSchema", "true") \
                      .load(info["path"])
    
    df_with_metadata = df.withColumn("data_ingestao", current_timestamp()) \
                         .withColumn("arquivo_origem", input_file_name())

    output_path = f"{bronze_path}{nome_tabela}"
    df_with_metadata.write.format("delta") \
                          .mode("overwrite") \
                          .save(output_path)
    
    print(f"Tabela '{nome_tabela}' salva com sucesso em '{output_path}'.")

print("\nIngestão para a camada Bronze concluída!")